<a href="https://colab.research.google.com/github/SciEcon/UTXO/blob/main/Deciphering_Bitcoin_Blockchain_Data_by_Cohort_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part I: Data Querying

## Preliminaries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import decimal
from datetime import datetime, date, timedelta, timezone

In [ ]:
#Connect to Google Cloud
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

In [ ]:
#Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
print('Authenticated')

In [ ]:
#Connect to Google BigQuery
PROJECT_ID = 'crypto-291811'

from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID, location='US')
dataset_ref = client.dataset('UTXO', project=PROJECT_ID)
dataset = client.get_dataset(dataset_ref)
tables = list(client.list_tables(dataset))

# Print names of all tables in the dataset
for table in tables:  
  print(table.table_id)

## Creating a Table for variables of interest

In [ ]:
#Create joint_all

table_id_inputs = "crypto-291811.UTXO.joint_all"
job_config = bigquery.QueryJobConfig(destination=table_id_inputs)

sql = """
  SELECT
    (outputs.value/POW(10,8)) AS UTXO,  
    outputs.block_timestamp,
    inputs.block_timestamp AS spent_block_timestamp,
    #FORMAT_TIMESTAMP("%Y-%m-%d", block_timestamp) AS block_date,
    #FORMAT_TIMESTAMP("%Y-%m-%d", spent_block_timestamp) AS spent_block_date,
  FROM 
    `bigquery-public-data.crypto_bitcoin.outputs` AS outputs
  LEFT JOIN 
    `bigquery-public-data.crypto_bitcoin.inputs` AS inputs
  ON outputs.transaction_hash=inputs.spent_transaction_hash  
  AND outputs.index = inputs.spent_output_index
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.


print("Query results loaded to the table {}".format(table_id_inputs))

## Create partitioned tables

In [ ]:
#Partition Table by born date for data after 2012

job_config = bigquery.QueryJobConfig()
sql = """
  CREATE TABLE
    `crypto-291811.UTXO.joint_all_partitionedbyborn12`
  PARTITION BY
    DATE(block_timestamp) AS
  SELECT
    *
  FROM
    `crypto-291811.UTXO.joint_all`
  WHERE
    block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id_inputs))

In [ ]:
#Partition by death date for data after 2012

job_config = bigquery.QueryJobConfig()
sql = """
  CREATE TABLE
    `crypto-291811.UTXO.joint_all_partitionedbydeath12`
  PARTITION BY
    DATE(spent_block_timestamp) AS
  SELECT
    *
  FROM
    `crypto-291811.UTXO.joint_all`
  WHERE
    (spent_block_timestamp > TIMESTAMP('2012-01-01 00:00:00+00')
    OR 
    spent_block_timestamp IS NULL)

"""

# Start the query, passing in the extra configuration.
query_job_inputs = client.query(sql, job_config=job_config)  # Make an API request.
query_job_inputs.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_id_inputs))

# Part II: Data Processing

## Define Functions

In [ ]:
def cal(x):
    t=np.sign(x-0.999)+np.sign(x-29.999)+np.sign(x-90.999)+np.sign(x-181.999)+np.sign(x-364.999)+np.sign(x-365*2+0.001)+np.sign(x-365*3+0.001)+np.sign(x-365*4+0.001)+np.sign(x-365*5+0.001)+np.sign(x-365*10+0.001)+1
    return t

def Task1_born(data):
    newborn = data['UTXO'].sum()
    return(newborn)

#Partitioning By Death Date
def Task1_dead(data):
    dead = data['UTXO'].sum()
    return(dead)

def Task2(data):
    #data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    #data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    sumUTXO = data['UTXO'].sum()
    sumLength = (data['UTXO']*data['Life_Length']).sum()
    if sumUTXO == 0:
        WALE = 0.0
    else:
        WALE = sumLength/sumUTXO
    return(WALE)
    
def Task3(data):
    data['Life_Length'] = data['spent_block_timestamp']- data['block_timestamp']
    data['Life_Length'] = data['Life_Length'].map(lambda x:x.days).apply(float)
    data['categorical'] = cal(data['Life_Length'])
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    for i in categories:  
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()  
    return result

def Task4(data, date):  
    categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
    result=pd.DataFrame(np.zeros((1, 11)), columns=categories)
    if len(data)!= 0:
      data['Age'] = data['block_timestamp'].apply(lambda x: (working_date-x).days)
      data['categorical'] = cal(data['Age'])
      for i in categories: 
        result.loc[:,i] = data[data['categorical']==i]['UTXO'].sum()
    return result

## Defining the Processing Programs

In [ ]:
def STXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Result=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Result['date'] = duration

  for i in range(0, days):
    start_date = start + timedelta(days=i)
    end_date = start_date + timedelta(days=1)
    
  #Partitioning by Dead Date
    query2 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all_partitionedbydeath12`
          WHERE
            spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            spent_block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
    query_job2 = client.query(query2)
    # Make an API request  to run the query and return a pandas DataFrame
    data2 = query_job2.to_dataframe()  
    
    #Work on Task3
    Result.iloc[i,0:11]=list(Task3(data2).iloc[0])
    
  #Partitioning by Born Date
    query1 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all_partitionedbyborn12`
          WHERE
            block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
          AND 
            block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
    query_job1 = client.query(query1)
    # Make an API request  to run the query and return a pandas DataFrame
    data1 = query_job1.to_dataframe()
    
    #Work on Task 1 and Task 2
    Result.loc[i,'newborn'] = Task1_born(data1)
    Result.loc[i,'dead'] = Task1_dead(data2)
    Result.loc[i,'WALE'] = Task2(data2)
    Result.columns = ['-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11', 'date', 'newborn', 'dead', 'WALE']
  return Result

In [ ]:
def UTXOprogram(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 10)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_partitionedbydeath12`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND 
        (spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
        OR 
        spent_block_timestamp IS NULL)
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:10] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram1(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_partitionedbydeath12`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND 
        spent_block_timestamp >= TIMESTAMP('""" + str(start_date) + """ 00:00:00+00')
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((data.spent_block_timestamp>=working_date)))].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram2(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_partitionedbydeath12`
      WHERE
        block_timestamp < TIMESTAMP('2018-12-31 00:00:00+00')
      AND 
        spent_block_timestamp IS NULL
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[(data.block_timestamp<working_date)].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

In [ ]:
def UTXOprogram3(start, end):
  duration=pd.date_range(start=start, end=end)
  days = np.size(duration)
  categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9, 11]
  Dist_Alive=pd.DataFrame(np.zeros((days, 11)), columns=categories)
  Dist_Alive['date'] = duration
  start_date=start+timedelta(days=1) 
  end_date =end+timedelta(days=1) 
  # note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
  #must be from joint_all
  query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all_partitionedbydeath12`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
      AND
        block_timestamp > TIMESTAMP('2018-12-31 00:00:00+00')
      AND 
        spent_block_timestamp IS NULL
     """
  query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
  data = query_job.to_dataframe()
  data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
  data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')
  for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[(data.block_timestamp<working_date)].copy()
    Dist_Alive.iloc[j,0:11] = list(Task4(working_data, working_date).iloc[0])

  return Dist_Alive

## Running the Processing Programs

In [ ]:
for year in range(2012, 2021):
  start = date(year,1,1)
  end = date(year,12,31)
  STXOresult = STXOprogram(start, end)
  address = '/content/drive/My Drive/ResultSTXO' + str(year) + '.csv'
  STXOresult.to_csv(address)

In [ ]:
for year in range(2012, 2021):
  start = date(year,1,1)
  end = date(year,12,31)
  UTXOresult = UTXOprogram(start, end)
  address = '/content/drive/My Drive/ResultUTXO' + str(year) + '.csv'
  STXOresult.to_csv(address)

Note: this is how we acquire results for all data from 2012 to 2020. In practice, you may adjust the start and end range to acquire results for any time period you want.

## 2009-2011 Data

In [ ]:
#STXO
end_date = date(2012,1,1)
query1 = """
          SELECT 
            *
          FROM 
            `crypto-291811.UTXO.joint_all`
          WHERE
            block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')"""
            
query_job1 = client.query(query1)

    # Make an API request  to run the query and return a pandas DataFrame
data = query_job1.to_dataframe()
data['UTXO'] = (data['value']/10**8).apply(float)
data = data.drop(['value'], axis = 1)

start = date(2009,1,3)
end = date(2011,12,31)
duration=pd.date_range(start=start, end=end)
days = np.size(duration)
categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9]
Result=pd.DataFrame(np.zeros((days, 10)), columns=categories)
Result['date'] = duration

for i in range(0, days):
    working_start_date = pd.to_datetime(start + timedelta(days=i),utc=True)
    working_end_date = pd.to_datetime(start +timedelta(days=1)+timedelta(days=i),utc=True)
    working_data2 = data[(data.spent_block_timestamp >= working_start_date) & (data.spent_block_timestamp < working_end_date)].copy()
    Result.iloc[i,0:10]=list(Task3(working_data2).iloc[0])
    working_data1 = data[(data.block_timestamp >= working_start_date) & (data.block_timestamp < working_end_date)].copy()
    Result.loc[i,'newborn'] = Task1_born(working_data1)
    Result.loc[i,'dead'] = Task1_dead(working_data2)
    Result.loc[i,'WALE'] = Task2(working_data2)
Result.to_csv('/content/drive/My Drive/ResultSTXO09-11.csv')

In [ ]:
start = date(2009,1,3)
end = date(2011,12,31)
duration=pd.date_range(start=start, end=end)
days = np.size(duration)
categories = [-9, -7, -5, -3, -1, 1, 3, 5, 7, 9]
Dist_Alive=pd.DataFrame(np.zeros((days, 10)), columns=categories)
Dist_Alive['date'] = duration

start_date=start+timedelta(days=1) 
end_date =end+timedelta(days=1) 
### note the trick below, we only keep data whose block_timestamp<end_date, and spent_block_timestamp>start_date
#must be from joint_all
query = """
      SELECT 
        *
      FROM 
        `crypto-291811.UTXO.joint_all`
      WHERE
        block_timestamp < TIMESTAMP('""" + str(end_date) + """ 00:00:00+00')
     """
query_job = client.query(query)

# Make an API request  to run the query and return a pandas DataFrame
data = query_job.to_dataframe()
data['block_timestamp'] = pd.to_datetime(data['block_timestamp'], format='%Y-%m-%d')
data['spent_block_timestamp'] = pd.to_datetime(data['spent_block_timestamp'], format='%Y-%m-%d')

for j in range(0, days):
    working_date = pd.to_datetime(start_date + timedelta(days=j), utc=True)   
    working_data = data.loc[((data.block_timestamp<working_date) & ((pd.isna(data.spent_block_timestamp) | (data.spent_block_timestamp>=working_date))))].copy()
    Dist_Alive.iloc[j,0:10] = list(Task4(working_data, working_date).iloc[0])

Dist_Alive.to_csv('/content/drive/My Drive/UTXO/DistAlive09-11.csv')

## Merge the Results and Export Data

In [ ]:
#Assuming that all necessary data files are ready in Google Drive
start = 2012
end = 2020
currency = "Bitcoin"

Task123 = pd.read_csv("/content/drive/My Drive/UTXO/ResultSTXO09-11.csv")
Dist_Alive = pd.read_csv("/content/drive/My Drive/UTXO/ResultUTXO09-11.csv")

for i in range(start, end+1):
  file1 = pd.read_csv('/content/drive/My Drive/UTXO/' + currency + "Result" + str(i) + "Task123.csv")
  file2 = pd.read_csv('/content/drive/My Drive/UTXO/' + currency + "DistAlive" + str(i) + ".csv")
  Task123 = Task123.append(file1)
  Dist_Alive = Dist_Alive.append(file2)

Task123 = Task123.reset_index(drop=True).drop(['Unnamed: 0'], axis = 1)
Task123 = Task123[['date', 'newborn', 'dead', 'WALE', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]
Dist_Alive = Dist_Alive.reset_index(drop=True).drop(['Unnamed: 0'], axis = 1)
Dist_Alive = Dist_Alive[['date', '-9', '-7', '-5', '-3', '-1', '1', '3', '5', '7', '9', '11']]

# Export Data
name1 = '/content/drive/My Drive/UTXO/' + currency + 'Task123.csv'
name2 = '/content/drive/My Drive/UTXO/' + currency + 'Distalive.csv'
Task123.to_csv(name1)
Dist_Alive.to_csv(name2)